In [1]:
# 필요한 모듈 불러오기
import pandas as pd
import numpy as np
import csv
import time

# 난수 발생 패턴을 고정하기 위해 난수는 seed(1234)로 설정
np.random.seed(1234)

# 고정되지 않는 난수를 발생하기 위해서 randomize함수를 이용해 현재시간을 기준으로 난수를 발생하는 함수
def randomize() :
    np.random.seed(time.time)

In [2]:
# 정규분포 난수값의 평균과 표준편차를 설정하여 가중치의 파라미터를 초기화 할때 사용한다
RND_MEAN = 0
RND_STD = 0.0030

# 학습률을 설정(하이퍼파라미터)
LEARNING_RATE = 0.001

In [ ]:
def abalone_exec(epoch_count=10, mb_size=10, report=1):
    load_abalone_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

def load_abalone_dataset():
    with open('abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader,None)
        rows = []
        for row in csvreader:
            rows.append(row)
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt+output_cnt])
    for n, row in enumerate(rows):
        if row[0] == 'I' : data[n, 0] = 1
        if row[0] == 'M' : data[n, 1] = 1
        if row[0] == 'F' : data[n, 2] = 1
        data[n, 3:] = row[1:]

def init_model_hidden1():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt

    pm_hidden = alloc_param_pair([input_cnt,hidden_cnt])
    pm_output = alloc_param_pair([hidden_cnt, output_cnt])
    
def alloc_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD,shape) 
    bias = np.zeros(shape[-1])
    return {'w':weight, 'b':bias}

def train_and_test(epoch_count, mb_size, report): 
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    for epoch in range(epoch_count):
        losses, accs = [], []
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss ={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

def forward_neuralnet_hidden1(x):
    global pm_output, pm_hidden

    hidden = relu(np.matmul(x,pm_hidden['w']) + pm_hidden['b'])
    output = np.matmul(hidden, output['w']) + pm_output['b']
    return output, [x,hidden]

def relu(x):
    return np.maximum(x, 0)

def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden

    x, hidden = aux

    g_output_w_out = hidden.transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis = 0)

    ##여기부터 추가작성 필요

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis = 0)
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

def backprop_postproc(G_loss, diff):
    shape = diff.shape
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1
    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    return G_output

def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y)/y))
    return 1- mdiff

In [5]:
abalone_exec(epoch_count=100, mb_size=10, report=1)

Epoch 1: loss =33.975, accuracy=0.555/0.817
Epoch 2: loss =8.253, accuracy=0.818/0.812
Epoch 3: loss =7.620, accuracy=0.810/0.807
Epoch 4: loss =7.520, accuracy=0.808/0.808
Epoch 5: loss =7.446, accuracy=0.808/0.807
Epoch 6: loss =7.381, accuracy=0.809/0.806
Epoch 7: loss =7.325, accuracy=0.808/0.807
Epoch 8: loss =7.275, accuracy=0.808/0.807
Epoch 9: loss =7.230, accuracy=0.807/0.810
Epoch 10: loss =7.196, accuracy=0.809/0.808
Epoch 11: loss =7.158, accuracy=0.809/0.805
Epoch 12: loss =7.130, accuracy=0.807/0.808
Epoch 13: loss =7.101, accuracy=0.809/0.807
Epoch 14: loss =7.075, accuracy=0.808/0.809
Epoch 15: loss =7.051, accuracy=0.809/0.809
Epoch 16: loss =7.028, accuracy=0.809/0.808
Epoch 17: loss =7.008, accuracy=0.808/0.809
Epoch 18: loss =6.988, accuracy=0.809/0.808
Epoch 19: loss =6.970, accuracy=0.809/0.808
Epoch 20: loss =6.953, accuracy=0.809/0.808
Epoch 21: loss =6.938, accuracy=0.808/0.809
Epoch 22: loss =6.921, accuracy=0.809/0.809
Epoch 23: loss =6.907, accuracy=0.809/0.